In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [1]:
import math
import random
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import  get_ground_state_for_g
from qsgenerator.states.simple_state_circuits import build_x_rotation_state
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r
from qsgenerator.evaluators.circuit_evaluator import CircuitEvaluator

In [2]:
generator_layers = 3
discriminator_layers = 5
data_bus_size = 3

In [3]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [4]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [5]:
real, real_symbols = build_x_rotation_state(qubits=data_qubits)

In [6]:
pure_gen = gen.copy()
gen.append([disc])

In [7]:
pure_real = real.copy()
real.append([disc])

In [8]:
print("REAL GROUND STATE")
print(pure_real)

REAL GROUND STATE
(0, 2): ───Rx(r0)───

(0, 3): ───Rx(r1)───

(0, 4): ───Rx(r2)───


In [9]:
print("GENERATOR")
print(pure_gen)

GENERATOR
(0, 2): ───Rz(l)───Rx(g0)───Rz(g4)───ZZ───────────────Rz(l)───Rx(g11)───Rz(g15)───ZZ────────────────Rz(l)───Rx(g22)───Rz(g26)───ZZ────────────────
                                     │                                            │                                             │
(0, 3): ───Rz(l)───Rx(g1)───Rz(g5)───ZZ^g8───ZZ───────Rz(l)───Rx(g12)───Rz(g16)───ZZ^g19───ZZ───────Rz(l)───Rx(g23)───Rz(g27)───ZZ^g30───ZZ───────
                                             │                                             │                                             │
(0, 4): ───Rz(l)───Rx(g2)───Rz(g6)───ZZ──────ZZ^g10───Rz(l)───Rx(g13)───Rz(g17)───ZZ───────ZZ^g21───Rz(l)───Rx(g24)───Rz(g28)───ZZ───────ZZ^g32───
                                     │                                            │                                             │
(0, 5): ───Rz(l)───Rx(g3)───Rz(g7)───ZZ^g9────────────Rz(l)───Rx(g14)───Rz(g18)───ZZ^g20────────────Rz(l)───Rx(g25)───Rz(g29)───ZZ^g31────────────


In [10]:
print("DISCRIMINATOR")
print(disc)

DISCRIMINATOR
(0, 0): ───Rz(l)───Rx(d0)───Rz(d5)───ZZ────────────────Rz(l)───Rx(d14)───Rz(d19)───ZZ────────────────Rz(l)───Rx(d28)───Rz(d33)───ZZ────────────────Rz(l)───Rx(d42)───Rz(d47)───ZZ────────────────Rz(l)───Rx(d56)───Rz(d61)───ZZ────────────────
                                     │                                             │                                             │                                             │                                             │
(0, 1): ───Rz(l)───Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rz(l)───Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────Rz(l)───Rx(d29)───Rz(d34)───ZZ^d38───ZZ───────Rz(l)───Rx(d43)───Rz(d48)───ZZ^d52───ZZ───────Rz(l)───Rx(d57)───Rz(d62)───ZZ^d66───ZZ───────
                                              │                                             │                                             │                                             │                                             │
(0, 2): ───Rz(l)───Rx(d2)───Rz(d7)───ZZ───────ZZ^d1

In [11]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [12]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [13]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [14]:
g_provider = lambda: random.choice([-1, 0])

In [15]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  use_analytical_expectation=True)

In [ ]:
epochs = 101
disc_iteration = 100
gen_iteration = 10
print_interval_epoch = 10
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      print_interval_epoch=print_interval_epoch)

In [20]:
def get_gen_for_g(g, gen_weights):
    rad = map_to_radians(g)
    return np.append(gen_weights, rad)

def get_variables_real(g):
    return np.array([get_theta_v(g), get_theta_w(g), get_theta_r(g)])

def get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, g, size):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, trained_gen_weights), list(range(size)))
    real = real_evaluator.get_state_from_params(get_variables_real(g))
    return generated, real, cirq.fidelity(generated, real)

def get_states_and_fidelty_for_ground(gen_evaluator, g, size):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, trained_gen_weights), list(range(size)))
    ground = get_ground_state_for_g(g, size)
    return generated, real, cirq.fidelity(generated, ground)

In [21]:
trained_disc_weights = tf.Variable(np.array([ 1.25888796e+01,  1.10409822e+01,  1.27487049e+01,  1.32927475e+01,
       -3.20522385e+01,  2.98508596e+00, -7.54223883e-01,  8.97036648e+00,
        8.98472309e+00, -2.77423954e+00,  8.90891266e+00,  5.72837019e+00,
        6.03105211e+00, -4.64482594e+00, -1.10843427e-01,  7.78598115e-02,
        3.00343895e+00,  3.83781940e-01,  6.08641243e+00,  8.64131927e+00,
       -2.17593918e+01,  1.45857897e+01,  2.36893883e+01,  8.75363445e+00,
       -2.42768993e+01,  1.27688437e+01,  2.53628349e+00,  1.39768391e+01,
        1.40961084e+01, -2.04474068e+01,  4.71392822e+00,  1.66926212e+01,
        1.88311214e+01,  9.17525005e+00,  1.16109962e+01,  1.08004580e+01,
        1.10795708e+01,  5.81477690e+00, -5.50215101e+00,  2.22007637e+01,
        2.25015125e+01,  3.21826210e+01, -1.80058708e+01, -7.85126591e+00,
       -7.77073908e+00,  1.40237570e+01,  3.14071465e+01, -1.07477732e+01,
        1.55852342e+00,  2.63786411e+01,  1.31890945e+01,  7.14759350e+00,
        5.46145630e+00, -1.79730053e+01,  7.40563774e+00,  1.06135674e+01,
       -1.55095673e+00, -2.66580944e+01,  6.51995277e+00, -2.37151980e+00,
        3.51896515e+01,  1.50027342e+01,  1.80733763e-02, -1.75052185e+01,
        9.60706902e+00, -6.06413984e+00,  5.35433817e+00, -7.88707399e+00,
        1.27363043e+01,  1.02446747e+01]), dtype=tf.float32)

trained_gen_weights = np.array([  0.83756703,  -8.85081   ,   5.5571547 ,   6.2937136 ,
       -11.173263  ,  -5.2586904 ,  15.195034  ,  -2.752592  ,
        -2.758591  , -11.47981   ,  -5.061309  ,   3.2534666 ,
        -5.6829824 , -22.536098  ,   9.015973  ,   0.5836532 ,
         2.1221178 ,   5.4982877 ,  13.235283  ,   3.4571006 ,
        -0.6003567 ,  -5.9493637 ,  -8.738516  ,   0.4041    ,
         6.9204884 ,  -6.442905  ,  -9.274843  ,  -3.695387  ,
        10.760881  ,   2.24611   ,   0.5289249 ,  14.787433  ,
       -11.227607])

gen_symbols = gs + (ls,)
real_symbols = ('r0', 'r1', 'r2')

In [22]:
gen_evaluator = CircuitEvaluator(pure_gen, gen_symbols)
real_evaluator = CircuitEvaluator(pure_real, real_symbols)

In [23]:
g = 1
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, g, data_bus_size)

Max probability state after tracing has probability: 0.981857419013977


(array([-0.0923465 +0.0000000e+00j, -0.07581477-4.6684846e-01j,
         0.08082028-1.9660823e-01j,  0.20732668+6.3416529e-01j,
         0.01657245+1.6675850e-02j, -0.07306691+1.8806756e-01j,
         0.14650305-2.5829076e-04j, -0.3853701 +2.5730491e-01j],
       dtype=complex64),
 array([ 0.60355335+0.j        ,  0.        -0.60355335j,
         0.        -0.24999999j, -0.24999999+0.j        ,
         0.        -0.24999999j, -0.24999999+0.j        ,
        -0.10355338+0.j        ,  0.        +0.10355338j], dtype=complex64),
 0.09694529183463008)

In [ ]:
g = 1
gen_evaluator.get_state_from_params(get_gen_for_g(g, trained_gen_weights), list(range(size)))

In [114]:
get_ground_state_for_g(g, size)

array([-0.41380294,  0.33104236,  0.33104236,  0.33104236,  0.33104236,
        0.33104236,  0.33104236, -0.41380294])

In [123]:
x_resolver = cirq.ParamResolver({'r0': get_theta_v(g), 'r1': get_theta_w(g), 'r2': get_theta_r(g)})
x_resolved = cirq.resolve_parameters(pure_real, x_resolver)

In [116]:
 cirq.final_state_vector(x_resolved)

array([ 0.2542495 +0.j        ,  0.        -0.2842596j ,
        0.        -0.56851923j, -0.63562375+0.j        ,
        0.        -0.09711468j, -0.10857751+0.j        ,
       -0.21715502+0.j        ,  0.        +0.24278669j], dtype=complex64)

In [124]:
 cirq.final_state_vector(x_resolved)

array([ 0.56851923+0.j        ,  0.        -0.63562375j,
        0.        -0.2542495j , -0.2842596 +0.j        ,
        0.        -0.21715502j, -0.24278669+0.j        ,
       -0.09711468+0.j        ,  0.        +0.10857751j], dtype=complex64)

In [125]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(g,size)))

1.5313699304113815

In [126]:
cirq.fidelity(highest_prob_state, cirq.final_state_vector(x_resolved))

0.07673630335638393